### File name             :  Single Augmentation
### Author                  :  Infosense Services
### Creation Date      :  25-Nov-2022
  
###  DATE                    |               BY               |    Version   |             DESCRIPTION
-- ------------------------------------------------------------------------------------------
### 01-Aug-2023        |      Vipul Prajapati    |      1.0         |        Initial Version Created 
### 04-Aug-2023        |      Vipul Prajapati    |      1.1         |        Optimize Code Use Py DF Instead DB Data

In [1]:
# Library Imports

import os
import gc
import ast
import json
import time
import uuid
import math
import traceback
import warnings
import urllib.parse
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras
warnings.filterwarnings("ignore")

In [2]:
# DATABASE PARAMETER

database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'

connetion = psycopg2.connect(database = database, user = user ,password = password, host = host,port = port)


# GLOBAL VARIABLE PARAMETER
volume_table = 'tm_new_data.nys_hour_8_volume_240626'
connected_ids_table = 'tm_new_data.ny_connected_osm_master_231211'
output_table = 'nys_augmentation_hour_8_volume_231211'


iteration = 2
source = 'SBA'
Missing_IDS = []
error_dict = {}

if source == 'SFA':tag ='next'
else:tag = 'prev' 
    
# augmented_df = pd.DataFrame({'isg_osm_id':[],'direction':[],'source':[],'highway_id':[],'volume':[],
#                              'original_ids':[],'processed_ids':[],'single_filled':[],'multi_filled':[],
#                              'iteration':[]},dtype='int64')

# selected_lst = ['isg_osm_id','direction','source','highway_id','volume',
#                'original_ids','processed_ids','single_filled','multi_filled','iteration']

# augmented_df = pd.DataFrame({'isg_osm_id':[],'direction':[],'source':[],'highway_id':[],'volume':[],
#                              'original_ids':[],'single_filled':[],'iteration':[]},dtype='int64')


# selected_lst = ['isg_osm_id','direction','source','highway_id','volume',
#                'original_ids','single_filled','iteration']

augmented_df = pd.DataFrame({'isg_osm_id':[],'source':[],'highway_id':[],'volume':[],
                             'original_ids':[],'single_filled':[],'iteration':[]},dtype='int64')


selected_lst = ['isg_osm_id','source','highway_id','volume',
               'original_ids','single_filled','iteration']


# GET BASE DATA FOR AVAILABLE VOLUME

# READ All BASE DATA

sql_command = f""" select distinct isg_osm_id,highway_id,region,volume 
                  from {volume_table} where volume > 0""" 

all_osm_data_df = pd.read_sql(sql_command, connetion)
ALL_Available_LST = np.unique(list(set(all_osm_data_df.to_dict(orient='list')['isg_osm_id']))).tolist()

    
sql_command = f"""select distinct isg_osm_id,highway_id,volume,source,
                         single_{tag}_ids as connected_ids from
                    (select A.*,B.single_{tag}_ids from 
                        (select * from {volume_table} where volume > 0 and {tag}_volume < 0 ) A
                        ,{connected_ids_table} B
                    where A.isg_osm_id = B.current_id
                    )foo where single_{tag}_ids !='{str('{}')}'""" 
    

# GET PROCESS LIST
All_Data_df = pd.read_sql(sql_command, connetion)

Updated_Available_LST = np.unique(list(set(All_Data_df.to_dict(orient='list')['isg_osm_id']))).tolist()

print("Available List :",len(ALL_Available_LST))
print("Total Available Data Length :",len(all_osm_data_df))
print("Process List :",len(Updated_Available_LST))
print("Total Process Data Length :",len(All_Data_df))
print("Number Of Columns :",All_Data_df.columns.tolist())
print(" ")

Available List : 62847
Total Available Data Length : 62847
Process List : 22487
Total Process Data Length : 22487
Number Of Columns : ['isg_osm_id', 'highway_id', 'volume', 'source', 'connected_ids']
 


In [ ]:
start = time.time()
counter = 0  
for search_osm_id in Updated_Available_LST: 

    try:
        original_id = search_osm_id
        # Extract Current Search IDS Data
        extract_df = All_Data_df.loc[All_Data_df['isg_osm_id'] == search_osm_id].reset_index(drop = True)
        single_level_ids_lst = extract_df['connected_ids'].iloc[0]
        unique_process_lst= single_level_ids_lst
        common_elements = set(single_level_ids_lst) & set(ALL_Available_LST)
        if len(common_elements) > 0:
            unique_process_lst = []
            for item in single_level_ids_lst:
                if item not in common_elements:
                    unique_process_lst.append(item) 
                else: 
                    break
     
        if len(unique_process_lst) > 0 :
            extract_df = extract_df.head(1)
            single_level_df = pd.DataFrame(np.repeat(extract_df.values, len(unique_process_lst), axis=0))
            single_level_df.columns = extract_df.columns
            single_level_df['isg_osm_id'] = unique_process_lst
            single_level_df['source'] = source
            single_level_df['original_ids'] = [original_id]
            #single_level_df['processed_ids'] = search_osm_id
            single_level_df['single_filled'] = str(unique_process_lst)
            #single_level_df['multi_filled'] = None
            single_level_df['iteration'] = iteration
            
            single_level_df= single_level_df[selected_lst]
            augmented_df=pd.concat([augmented_df, single_level_df], ignore_index=True)
            
        counter = counter + 1
        print(counter)
        
    except:
        error_dict[str(search_osm_id)] = str(traceback.format_exc())
        Missing_IDS.append(search_osm_id)
        continue

end = time.time()

17011
17012
17013
17014
17015
17016
17017
17018
17019
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
17033
17034
17035
17036
17037
17038
17039
17040
17041
17042
17043
17044
17045
17046
17047
17048
17049
17050
17051
17052
17053
17054
17055
17056
17057
17058
17059
17060
17061
17062
17063
17064
17065
17066
17067
17068
17069
17070
17071
17072
17073
17074
17075
17076
17077
17078
17079
17080
17081
17082
17083
17084
17085
17086
17087
17088
17089
17090
17091
17092
17093
17094
17095
17096
17097
17098
17099
17100
17101
17102
17103
17104
17105
17106
17107
17108
17109
17110
17111
17112
17113
17114
17115
17116
17117
17118
17119
17120
17121
17122
17123
17124
17125
17126
17127
17128
17129
17130
17131
17132
17133
17134
17135
17136
17137
17138
17139
17140
17141
17142
17143
17144
17145
17146
17147
17148
17149
17150
17151
17152
17153
17154
17155
17156
17157
17158
17159
17160
17161
17162
17163
17164
17165
17166
17167
17168
17169
17170
17171
17172
17173
17174
17175
17176
1717

18395
18396
18397
18398
18399
18400
18401
18402
18403
18404
18405
18406
18407
18408
18409
18410
18411
18412
18413
18414
18415
18416
18417
18418
18419
18420
18421
18422
18423
18424
18425
18426
18427
18428
18429
18430
18431
18432
18433
18434
18435
18436
18437
18438
18439
18440
18441
18442
18443
18444
18445
18446
18447
18448
18449
18450
18451
18452
18453
18454
18455
18456
18457
18458
18459
18460
18461
18462
18463
18464
18465
18466
18467
18468
18469
18470
18471
18472
18473
18474
18475
18476
18477
18478
18479
18480
18481
18482
18483
18484
18485
18486
18487
18488
18489
18490
18491
18492
18493
18494
18495
18496
18497
18498
18499
18500
18501
18502
18503
18504
18505
18506
18507
18508
18509
18510
18511
18512
18513
18514
18515
18516
18517
18518
18519
18520
18521
18522
18523
18524
18525
18526
18527
18528
18529
18530
18531
18532
18533
18534
18535
18536
18537
18538
18539
18540
18541
18542
18543
18544
18545
18546
18547
18548
18549
18550
18551
18552
18553
18554
18555
18556
18557
18558
18559
18560
1856

19766
19767
19768
19769
19770
19771
19772
19773
19774
19775
19776
19777
19778
19779
19780
19781
19782
19783
19784
19785
19786
19787
19788
19789
19790
19791
19792
19793
19794
19795
19796
19797
19798
19799
19800
19801
19802
19803
19804
19805
19806
19807
19808
19809
19810
19811
19812
19813
19814
19815
19816
19817
19818
19819
19820
19821
19822
19823
19824
19825
19826
19827
19828
19829
19830
19831
19832
19833
19834
19835
19836
19837
19838
19839
19840
19841
19842
19843
19844
19845
19846
19847
19848
19849
19850
19851
19852
19853
19854
19855
19856
19857
19858
19859
19860
19861
19862
19863
19864
19865
19866
19867
19868
19869
19870
19871
19872
19873
19874
19875
19876
19877
19878
19879
19880
19881
19882
19883
19884
19885
19886
19887
19888
19889
19890
19891
19892
19893
19894
19895
19896
19897
19898
19899
19900
19901
19902
19903
19904
19905
19906
19907
19908
19909
19910
19911
19912
19913
19914
19915
19916
19917
19918
19919
19920
19921
19922
19923
19924
19925
19926
19927
19928
19929
19930
19931
1993

21137
21138
21139
21140
21141
21142
21143
21144
21145
21146
21147
21148
21149
21150
21151
21152
21153
21154
21155
21156
21157
21158
21159
21160
21161
21162
21163
21164
21165
21166
21167
21168
21169
21170
21171
21172
21173
21174
21175
21176
21177
21178
21179
21180
21181
21182
21183
21184
21185
21186
21187
21188
21189
21190
21191
21192
21193
21194
21195
21196
21197
21198
21199
21200
21201
21202
21203
21204
21205
21206
21207
21208
21209
21210
21211
21212
21213
21214
21215
21216
21217
21218
21219
21220
21221
21222
21223
21224
21225
21226
21227
21228
21229
21230
21231
21232
21233
21234
21235
21236
21237
21238
21239
21240
21241
21242
21243
21244
21245
21246
21247
21248
21249
21250
21251
21252
21253
21254
21255
21256
21257
21258
21259
21260
21261
21262
21263
21264
21265
21266
21267
21268
21269
21270
21271
21272
21273
21274
21275
21276
21277
21278
21279
21280
21281
21282
21283
21284
21285
21286
21287
21288
21289
21290
21291
21292
21293
21294
21295
21296
21297
21298
21299
21300
21301
21302
2130

In [ ]:
print("TIME TAKEN :",(end - start))

In [ ]:
print(len(augmented_df['original_ids'].unique()))
print(len(augmented_df['isg_osm_id'].unique()))
print(len(augmented_df['isg_osm_id']))
print(len(Missing_IDS))
print(len(error_dict.keys()))

In [ ]:
augmented_df.head()

In [ ]:
# Insert Data Into DB

database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'

encoded_password = urllib.parse.quote(password)

schema_name = 'tm_new_data'
table_name = output_table

connection_string = "postgresql://" + user + ":" + encoded_password + "@" + host + ":" + port + "/" + database
engine = create_engine(connection_string)

augmented_df.to_sql(table_name,engine,if_exists='append',schema=schema_name, chunksize=10000,index=False,method='multi')

In [ ]:
del [[All_Data_df,Updated_Available_LST,augmented_df,all_osm_data_df,ALL_Available_LST]]
gc.collect()